In [2]:
import pandas as pd
import boto3, json, time
from datetime import datetime, timedelta

In [5]:
client = boto3.client('logs')

last_x_days = 300
log_group_names = [
    "/aws/sagemaker/Endpoints/amazon-falconlite2-2023-10-07-18-07-48-251", 
    # "/aws/sagemaker/Endpoints/MistralLite-2023-10-19-09-58-03",
]

In [14]:
def query_cw_logs(query, last_x_days=last_x_days, log_group_names=log_group_names):
    last_time = timedelta(days=last_x_days)

    start_query_response = client.start_query(
        logGroupNames=log_group_names,
        startTime=int((datetime.today() - last_time).timestamp()),
        endTime=int(datetime.now().timestamp()),
        queryString=query,
    )
    
    query_id = start_query_response['queryId']
    response = None
    while response == None or response['status'] == 'Running':
        print('Waiting 2 secs for query to complete ...')
        time.sleep(2)
        response = client.get_query_results(
            queryId=query_id
        )
    print('Done!')
    # response.keys(): dict_keys(['results', 'statistics', 'status', 'ResponseMetadata'])
    results_formatted = [{item["field"]: item["value"] for item in list_dict} for list_dict in response["results"]]
    response["results"] = results_formatted
    return response 
    # data = []
    # for item in response["results"]:
    #     new_item = {}
    #     for record in item:
    #         new_item[record["field"]] = record["value"]
    #     data.append(new_item)
    
    # df = pd.DataFrame(data)
    # return df

# TODO: have the llm return - query, time_period, log_groups as well
# df = query_cw_logs(query, last_x_days=300)
# df.head()

In [15]:
# get all records with ERROR sort with timestamp and log_group_id
# query = """
# fields @message 
# | filter @message like /ERROR/
# | parse @message "* : *" as loggingRandom, loggingMessage
# | display @timestamp, loggingMessage
# | sort @timestamp 
# """

# Extracts the fields loggingTime, loggingType and loggingMessage, filters down to log events that contain ERROR or INFO strings, and then displays only the loggingMessage and loggingType fields for events that contain an ERROR string.
# query = """
# fields @message
#     | parse @message "* [*] *" as loggingTime, loggingType, loggingMessage
#     | filter loggingType IN ["ERROR", "INFO"]
#     | display loggingMessage, loggingType = "ERROR" as isError
# """

# "show the latest 20 log events",
query = """fields @timestamp, @message
| sort @timestamp desc
| limit 20"""

In [16]:
response = query_cw_logs(query, last_x_days=300)

Waiting 2 secs for query to complete ...
Waiting 2 secs for query to complete ...
Done!


In [22]:
# response.keys()

response["results"][0]

{'@timestamp': '2023-10-07 18:16:51.277',
 '@message': 'Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.',
 '@ptr': 'CpQBClQKUDMyNDYyMjQwMDUxNDovYXdzL3NhZ2VtYWtlci9FbmRwb2ludHMvYW1hem9uLWZhbGNvbmxpdGUyLTIwMjMtMTAtMDctMTgtMDctNDgtMjUxEAQSOBoYAgZPOTnzAAAAAnuR8+YABlIaBBAAAAAyIAEo5IXo2rAxMM3+7NqwMTg5QLnTCUjbpQFQh6IBGAAgARA4GAE='}

In [23]:
for item in response["results"]:
    print(item["@message"])

Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
RuntimeError: CUDA error: invalid argument
> File "/opt/conda/lib/python3.9/site-packages/text_generation_server/interceptor.py", line 21, in intercept
    return await response
  File "/opt/conda/lib/python3.9/site-packages/opentelemetry/instrumentation/grpc/_aio_server.py", line 82, in _unary_interceptor
    raise error
  File "/opt/conda/lib/python3.9/site-packages/opentelemetry/instrumentation/grpc/_aio_server.py", line 73, in _unary_interceptor
    return await behavior(request_or_iterator, context)
  File "/opt/conda/lib/python3.9/site-packages/text_generation_server/server.py", line 121, in Decode
    generations, next_batch = self.model.generate_token(batch)
  File "/opt/c

In [25]:
results = response["results"]
%store results

Stored 'results' (list)
